将原始数据转换为预处理数据：csv-->xlsx；时间列处理

In [1]:
import os
import subprocess
import pandas as pd
import sys
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter, column_index_from_string

folder_path = r'D:\Research\MarmoCo24fall\Data_B8\ToFilter'  # replace with your actual folder path
print(folder_path)

# Get the current directory
current_dir = os.getcwd()

D:\Research\MarmoCo24fall\Data_B8\ToFilter


将csv数据转置并储存为xlsx

In [2]:
dir_path = folder_path  
def convert_csv_to_xlsx(dir_path):
    # Loop through each file in the directory
    for filename in os.listdir(dir_path):
        if filename.endswith('.csv'):
            # Read the CSV data into a DataFrame without a header
            df = pd.read_csv(os.path.join(dir_path, filename), header=None)

            # Transpose the DataFrame
            df_transposed = df.transpose()

            # Save the transposed DataFrame to an Excel file without column names
            df_transposed.to_excel(os.path.join(dir_path, f'{os.path.splitext(filename)[0]}.xlsx'), index=False, header=False)

convert_csv_to_xlsx(dir_path)


插入times、timem、t0、t1和note列

In [3]:

def modify_xlsx_files(folder_path):
    output_folder = os.path.join(folder_path, 'precl')
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    xlsx_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
    for xlsx_file in xlsx_files:
        file_path = os.path.join(folder_path, xlsx_file)
        wb = openpyxl.load_workbook(file_path)
        ws = wb.active

        time_column = None
        button_column = None
        for col in ws.iter_cols(min_col=1, max_col=ws.max_column):
            for cell in col:
                if cell.value == 'time':
                    time_column = cell.column_letter
                if cell.value == 'button':
                    button_column = cell.column_letter
            if time_column and button_column:
                break

        if not time_column:
            print(f'在文件 {xlsx_file} 中找不到 "time" 列。跳过该文件。')
            continue

        if not button_column:
            print(f'在文件 {xlsx_file} 中找不到 "button" 列。跳过该文件。')
            continue

        # 插入 "times" 列和 "timem" 列
        time_column_index = column_index_from_string(time_column)
        ws.insert_cols(time_column_index + 1, 4)  # Inserting 4 columns instead of 2
        ws.cell(row=1, column=time_column_index + 1).value = 'times'
        ws.cell(row=1, column=time_column_index + 2).value = 'timem'
        ws.cell(row=1, column=time_column_index + 3).value = 't0'  # New column name
        ws.cell(row=1, column=time_column_index + 4).value = 't1'  # New column name

        for row in range(2, ws.max_row + 1):
            time_cell = ws.cell(row=row, column=time_column_index).value

            # 检查 "time" 列的值是否为 None 或空字符串
            if time_cell is None or time_cell == '':
                print(f'在文件 {xlsx_file} 第 {row} 行的 "time" 列中存在空值。跳过该行。')
                continue

            try:
                # 进行后续处理
                time_cell = int(time_cell)  # 将 time_cell 转换为整数类型
            except ValueError:
                print(f'在文件 {xlsx_file} 第 {row} 行的 "time" 列中存在无效值。跳过该行。')
                continue

            times_value = time_cell / 1000
            timem_value = time_cell / 60000

            ws.cell(row=row, column=time_column_index + 1).value = times_value
            ws.cell(row=row, column=time_column_index + 2).value = timem_value
            ws.cell(row=row, column=time_column_index + 3).value = ''  # Leave it empty for "t0"
            ws.cell(row=row, column=time_column_index + 4).value = ''  # Leave it empty for "t1"
    
        # 获取最后一列的索引
        last_column_index = ws.max_column

        # 插入 "note" 列
        note_column_index = last_column_index + 1  # Insert "note" column at the end of the chart
        ws.insert_cols(note_column_index)
        ws.cell(row=1, column=note_column_index).value = 'note'

        # 找出 'button' 列中第一次出现 'correct' 的行
        button_column_index = column_index_from_string(button_column)
        correct_row = None
        for row in range(2, ws.max_row + 1):
            if ws.cell(row=row, column=button_column_index).value == 'correct':
                correct_row = row
                break

        # 在 'note' 列中写入 'exp' 对于 'correct' 行及其之前的所有行
        if correct_row:
            for row in range(2, correct_row + 1):
                ws.cell(row=row, column=note_column_index).value = 'exp'

        # 保存 XLSX 文件
        output_file = os.path.join(output_folder, xlsx_file)
        wb.save(output_file)

        print(f'修改完成：{output_file}')

    print('所有 XLSX 文件已修改。')

modify_xlsx_files(folder_path)

在文件 1101.xlsx 第 460 行的 "time" 列中存在空值。跳过该行。
修改完成：D:\Research\MarmoCo24fall\Data_B8\ToFilter\precl\1101.xlsx
在文件 1102.xlsx 第 303 行的 "time" 列中存在空值。跳过该行。
修改完成：D:\Research\MarmoCo24fall\Data_B8\ToFilter\precl\1102.xlsx
在文件 1105.xlsx 第 500 行的 "time" 列中存在空值。跳过该行。
修改完成：D:\Research\MarmoCo24fall\Data_B8\ToFilter\precl\1105.xlsx
所有 XLSX 文件已修改。
